In [1]:
# ====================================================================
# 건강행태와 만성질환 상관관계 분석
# 서울시 지역사회건강조사 데이터 - Google Colab용
# ====================================================================

# 1. 라이브러리 설치 및 임포트
print("📦 라이브러리 설치 중...")
!pip install plotly kaleido -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

print("✅ 라이브러리 준비 완료!")

# ====================================================================
# 2. 데이터 로드
# ====================================================================
print("\n📂 데이터 로딩...")
from google.colab import files
print("CSV 파일을 업로드해주세요:")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, encoding='cp949')
print(f"✅ 데이터 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}열")
print(f"\n처음 3행:")
print(df.head(3))

# ====================================================================
# 3. 변수 매핑 및 전처리
# ====================================================================
print("\n🔧 데이터 전처리 중...")

# 변수명 매핑
var_mapping = {
    'age': '만나이',
    'gender': '성별',
    'hypertension': '고혈압 진단 경험 여부',
    'diabetes': '당뇨병 진단 경험 여부',
    'hyper_med': '고혈압 약 복용 여부',
    'stress': '주관적 스트레스 수준',
    'depression': '우울감 경험 여부',
    'sleep_hours': '하루 평균 수면시간 중',
    'smoking': '일반담배 현재흡연 여부',
    'drinking_freq': '연간 음주 빈도',
    'high_exercise': '고강도활동 일수',
    'moderate_exercise': '중강도활동 일수',
    'walk_days': '10분걷기 일수',
    'breakfast': '주간 아침식사',
    'height': '키',
    'weight': '몸무게',
}

rename_dict = {v: k for k, v in var_mapping.items() if v in df.columns}
df_clean = df.rename(columns=rename_dict).copy()

# 결측값 처리 (8, 88, 888 등을 NaN으로)
missing_codes = [8, 88, 888, 8888, 88888]
for col in df_clean.select_dtypes(include=[np.number]).columns:
    df_clean[col] = df_clean[col].replace(missing_codes, np.nan)

# BMI 계산
if 'height' in df_clean.columns and 'weight' in df_clean.columns:
    df_clean['bmi'] = df_clean['weight'] / ((df_clean['height'] / 100) ** 2)

# 만성질환 변수 생성
if 'hypertension' in df_clean.columns:
    df_clean['has_hypertension'] = (df_clean['hypertension'] == 1).astype(int)
if 'diabetes' in df_clean.columns:
    df_clean['has_diabetes'] = (df_clean['diabetes'] == 1).astype(int)

df_clean['has_chronic'] = (
    (df_clean.get('has_hypertension', pd.Series([0])) == 1) |
    (df_clean.get('has_diabetes', pd.Series([0])) == 1)
).astype(int)

# 정신건강 변수
if 'stress' in df_clean.columns:
    df_clean['stress_level'] = 5 - df_clean['stress']  # 높을수록 스트레스 높음

if 'depression' in df_clean.columns:
    df_clean['has_depression'] = (df_clean['depression'] == 1).astype(int)

df_clean['good_mental_health'] = (
    (df_clean.get('stress_level', 3) <= 2) &
    (df_clean.get('has_depression', pd.Series([0])) == 0)
).astype(int)

# 운동 점수 계산
exercise_cols = ['high_exercise', 'moderate_exercise', 'walk_days']
available_exercise = [col for col in exercise_cols if col in df_clean.columns]
if available_exercise:
    df_clean['exercise_score'] = df_clean[available_exercise].fillna(0).sum(axis=1)

# 연령대 구분
if 'age' in df_clean.columns:
    df_clean['age_group'] = pd.cut(
        df_clean['age'],
        bins=[0, 30, 40, 50, 60, 70, 100],
        labels=['20대이하', '30대', '40대', '50대', '60대', '70대이상']
    )

# 성별 라벨
if 'gender' in df_clean.columns:
    df_clean['gender_label'] = df_clean['gender'].map({1: '남성', 2: '여성'})

print(f"✅ 전처리 완료! 최종 데이터: {df_clean.shape[0]:,}행")

# ====================================================================
# 4. 핵심 분석 1: 만성질환 유병률
# ====================================================================
print("\n" + "="*70)
print("📈 분석 1: 만성질환 유병률")
print("="*70)

chronic_total = df_clean['has_chronic'].sum()
chronic_rate = df_clean['has_chronic'].mean() * 100

print(f"\n전체 대상자: {len(df_clean):,}명")
print(f"고혈압 보유: {df_clean.get('has_hypertension', pd.Series([0])).sum():,}명")
print(f"당뇨병 보유: {df_clean.get('has_diabetes', pd.Series([0])).sum():,}명")
print(f"만성질환 보유(중복포함): {chronic_total:,}명 ({chronic_rate:.1f}%)")

# 연령대별 유병률
if 'age_group' in df_clean.columns:
    age_chronic = df_clean.groupby('age_group')['has_chronic'].agg(['sum', 'mean'])
    age_chronic.columns = ['보유자수', '유병률']
    age_chronic['유병률'] = (age_chronic['유병률'] * 100).round(1)
    print("\n연령대별 만성질환 유병률:")
    print(age_chronic)

# 성별 유병률
if 'gender_label' in df_clean.columns:
    gender_chronic = df_clean.groupby('gender_label')['has_chronic'].mean() * 100
    print("\n성별 유병률(%):")
    print(gender_chronic.round(1))

# ====================================================================
# 5. 핵심 분석 2: 건강행태-만성질환 상관관계
# ====================================================================
print("\n" + "="*70)
print("📊 분석 2: 건강행태-만성질환 상관관계")
print("="*70)

# 상관분석할 변수 선택
correlation_vars = ['has_chronic', 'stress_level', 'exercise_score', 'bmi', 'age']
available_vars = [v for v in correlation_vars if v in df_clean.columns]

if len(available_vars) >= 2:
    # 상관계수 계산
    corr_data = df_clean[available_vars].dropna()
    corr_matrix = corr_data.corr()

    print(f"\n분석 대상: {len(corr_data):,}명 (결측값 제외)")
    print("\n상관계수 매트릭스:")
    print(corr_matrix.round(3))

    # 만성질환과의 상관관계 정렬
    if 'has_chronic' in corr_matrix.columns:
        chronic_corr = corr_matrix['has_chronic'].drop('has_chronic').abs().sort_values(ascending=False)
        print("\n만성질환과 가장 높은 상관관계:")
        for var, corr in chronic_corr.head(3).items():
            print(f"  - {var}: {corr:.3f}")

    # 히트맵 시각화
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale='RdBu_r',
        zmid=0,
        text=corr_matrix.values.round(2),
        texttemplate='%{text}',
        textfont={"size": 12},
    ))

    fig.update_layout(
        title='건강행태-만성질환 상관관계 히트맵',
        width=700,
        height=600,
        xaxis_title="",
        yaxis_title=""
    )

    fig.show()
else:
    print("⚠️ 상관분석에 필요한 변수가 부족합니다.")

# ====================================================================
# 6. 핵심 분석 3: 긍정 사례 (만성질환 + 정신건강 양호)
# ====================================================================
print("\n" + "="*70)
print("✨ 분석 3: 긍정 사례 - 만성질환이 있지만 정신건강 양호")
print("="*70)

# 건강 프로필 4그룹 분류
df_clean['health_profile'] = '기타'
df_clean.loc[df_clean['has_chronic'] == 0, 'health_profile'] = '만성질환없음'
df_clean.loc[
    (df_clean['has_chronic'] == 1) & (df_clean['good_mental_health'] == 1),
    'health_profile'
] = '긍정사례'
df_clean.loc[
    (df_clean['has_chronic'] == 1) & (df_clean['good_mental_health'] == 0),
    'health_profile'
] = '위험군'

profile_counts = df_clean['health_profile'].value_counts()
print("\n건강 프로필 분포:")
for profile, count in profile_counts.items():
    pct = count / len(df_clean) * 100
    print(f"  {profile}: {count:,}명 ({pct:.1f}%)")

# 긍정사례와 위험군 비교
positive_group = df_clean[df_clean['health_profile'] == '긍정사례']
risk_group = df_clean[df_clean['health_profile'] == '위험군']

if len(positive_group) > 0 and len(risk_group) > 0:
    print(f"\n긍정사례: {len(positive_group):,}명")
    print(f"위험군: {len(risk_group):,}명")

    comparison_vars = ['exercise_score', 'bmi', 'age']
    available_comp = [v for v in comparison_vars if v in df_clean.columns]

    if available_comp:
        print("\n긍정사례 vs 위험군 비교:")
        comparison_df = pd.DataFrame({
            '긍정사례': positive_group[available_comp].mean(),
            '위험군': risk_group[available_comp].mean(),
            '차이(긍정-위험)': positive_group[available_comp].mean() - risk_group[available_comp].mean()
        }).round(2)
        print(comparison_df)

        # 통계적 유의성 검정 (t-test)
        print("\n통계적 유의성 검정 (t-test):")
        for var in available_comp:
            pos_data = positive_group[var].dropna()
            risk_data = risk_group[var].dropna()
            if len(pos_data) > 0 and len(risk_data) > 0:
                t_stat, p_value = stats.ttest_ind(pos_data, risk_data)
                sig = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                print(f"  {var}: p={p_value:.4f} {sig}")
else:
    print("⚠️ 긍정사례 또는 위험군 데이터가 부족합니다.")

# ====================================================================
# 7. 종합 시각화 대시보드
# ====================================================================
print("\n" + "="*70)
print("📊 인터랙티브 대시보드 생성 중...")
print("="*70)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '연령대별 만성질환 유병률',
        '건강 프로필 분포',
        '그룹별 운동점수 비교',
        '긍정사례 vs 위험군 특성'
    ),
    specs=[
        [{'type': 'bar'}, {'type': 'pie'}],
        [{'type': 'box'}, {'type': 'bar'}]
    ]
)

# 차트 1: 연령대별 유병률
if 'age_group' in df_clean.columns:
    age_prev = df_clean.groupby('age_group')['has_chronic'].mean() * 100
    fig.add_trace(
        go.Bar(
            x=age_prev.index.astype(str),
            y=age_prev.values,
            marker_color='steelblue',
            name='유병률'
        ),
        row=1, col=1
    )

# 차트 2: 건강 프로필 파이차트
colors = ['#2ecc71', '#e74c3c', '#95a5a6', '#3498db']
fig.add_trace(
    go.Pie(
        labels=profile_counts.index,
        values=profile_counts.values,
        marker=dict(colors=colors),
        hole=0.3
    ),
    row=1, col=2
)

# 차트 3: 운동점수 박스플롯
if 'exercise_score' in df_clean.columns:
    for profile in ['긍정사례', '위험군', '만성질환없음']:
        data = df_clean[df_clean['health_profile'] == profile]['exercise_score'].dropna()
        if len(data) > 0:
            fig.add_trace(
                go.Box(y=data, name=profile, boxmean='sd'),
                row=2, col=1
            )

# 차트 4: 그룹별 특성 비교
if len(positive_group) > 0 and len(risk_group) > 0 and available_comp:
    comparison_data = pd.DataFrame({
        '긍정사례': positive_group[available_comp].mean(),
        '위험군': risk_group[available_comp].mean()
    })

    for col in comparison_data.columns:
        fig.add_trace(
            go.Bar(x=comparison_data.index, y=comparison_data[col], name=col),
            row=2, col=2
        )

fig.update_layout(
    title_text="건강행태-만성질환 분석 대시보드",
    height=900,
    showlegend=True
)

fig.show()
print("✅ 대시보드 생성 완료!")

# ====================================================================
# 8. 주요 인사이트 요약
# ====================================================================
print("\n" + "="*70)
print("💡 주요 인사이트")
print("="*70)

insights = []

# 1. 전체 유병률
chronic_rate = df_clean['has_chronic'].mean() * 100
insights.append(f"1. 전체 대상자의 {chronic_rate:.1f}%가 만성질환을 보유하고 있습니다.")

# 2. 긍정사례 비율
if len(positive_group) > 0:
    positive_rate = len(positive_group) / len(df_clean) * 100
    positive_of_chronic = len(positive_group) / df_clean['has_chronic'].sum() * 100
    insights.append(f"2. 만성질환 보유자 중 {positive_of_chronic:.1f}%가 정신건강이 양호한 '긍정사례'입니다.")

# 3. 운동의 영향
if len(positive_group) > 0 and len(risk_group) > 0 and 'exercise_score' in df_clean.columns:
    exercise_diff = positive_group['exercise_score'].mean() - risk_group['exercise_score'].mean()
    insights.append(f"3. 긍정사례는 위험군보다 평균 {abs(exercise_diff):.1f}일 {'더 많이' if exercise_diff > 0 else '더 적게'} 운동합니다.")

# 4. 주요 상관요인
if len(available_vars) > 2 and 'has_chronic' in corr_matrix.columns:
    top_factor = chronic_corr.index[0]
    top_corr = chronic_corr.iloc[0]
    insights.append(f"4. 만성질환과 가장 높은 상관관계: {top_factor} (r={top_corr:.3f})")

# 5. 연령 영향
if 'age_group' in df_clean.columns:
    oldest_prev = df_clean.groupby('age_group')['has_chronic'].mean().max() * 100
    insights.append(f"5. 고연령층의 만성질환 유병률은 최대 {oldest_prev:.1f}%입니다.")

print()
for insight in insights:
    print(f"  {insight}")

# ====================================================================
# 9. 결과 저장
# ====================================================================
print("\n" + "="*70)
print("💾 분석 결과 저장")
print("="*70)

# 긍정사례 샘플 저장
if len(positive_group) > 0:
    save_cols = ['age', 'gender_label', 'exercise_score', 'bmi', 'stress_level']
    available_save = [col for col in save_cols if col in positive_group.columns]

    if available_save:
        positive_sample = positive_group[available_save].head(100)
        positive_sample.to_csv('긍정사례_샘플.csv', index=False, encoding='utf-8-sig')
        print(f"✅ 긍정사례_샘플.csv 저장 완료 ({len(positive_sample)}명)")

# 건강프로필별 요약통계
if available_comp:
    summary_stats = df_clean.groupby('health_profile')[available_comp].agg(['mean', 'std']).round(2)
    summary_stats.to_csv('건강프로필_요약통계.csv', encoding='utf-8-sig')
    print("✅ 건강프로필_요약통계.csv 저장 완료")

# 상관계수 저장
if len(available_vars) >= 2:
    corr_matrix.to_csv('상관계수_매트릭스.csv', encoding='utf-8-sig')
    print("✅ 상관계수_매트릭스.csv 저장 완료")

print("\n" + "="*70)
print("🎉 분석 완료!")
print("="*70)
print("\n추천 후속 분석:")
print("  1. 로지스틱 회귀분석으로 만성질환 위험요인 예측 모델 구축")
print("  2. 군집분석(K-means)으로 건강행태 유형 분류")
print("  3. 지역구별, 소득수준별 건강 격차 분석")
print("  4. 긍정사례의 세부 생활습관 패턴 심층 분석")
print("\n파일 다운로드는 좌측 폴더 아이콘에서 확인하세요!")

📦 라이브러리 설치 중...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.4 MB/s eta 0:00:00
✅ 라이브러리 준비 완료!

📂 데이터 로딩...
CSV 파일을 업로드해주세요:


Saving 서울2024_변수가공.csv to 서울2024_변수가공.csv
✅ 데이터 로드 완료: 22,906행 × 66열

처음 3행:
       조사대상자 번호  만나이  성별  행정기관코드  주택유형  전체가구원 수  세대유형 (1~7)  기초생활수급자여부  \
0  2.024110e+24   31   1   11440     2        2           2          3   
1  2.024110e+24   29   2   11440     2        2           2          3   
2  2.024110e+24   31   1   11440     2        1           1          3   

   주관적 건강수준  평생 담배 제품 사용 경험  평생 음주 여부  전체가구소득_(월)금액  고강도활동 일수  고강도활동 전체시간(분)  \
0         2               2         1          1000         3             80   
1         2               2         1          1000         0              0   
2         2               1         1          1000         2             60   

   중강도활동 일수  중강도활동 전체시간(분)  근력일수  10분걷기 일수  10분걷기 전체시간(분)  주간 아침식사      키  \
0         0              0     4         7             60        2  181.0   
1         0              0     1         7             60        4  162.0   
2         2             60     3         5             60        4  176.0 


✨ 분석 3: 긍정 사례 - 만성질환이 있지만 정신건강 양호

건강 프로필 분포:
  만성질환없음: 15,811명 (69.0%)
  긍정사례: 5,398명 (23.6%)
  위험군: 1,697명 (7.4%)

긍정사례: 5,398명
위험군: 1,697명

긍정사례 vs 위험군 비교:
                 긍정사례     위험군  차이(긍정-위험)
exercise_score   7.08    6.37       0.71
bmi             83.09  161.66     -78.56
age             67.43   63.28       4.15

통계적 유의성 검정 (t-test):
  exercise_score: p=0.0000 ***
  bmi: p=0.0977 ns
  age: p=0.0000 ***

📊 인터랙티브 대시보드 생성 중...


✅ 대시보드 생성 완료!

💡 주요 인사이트

  1. 전체 대상자의 31.0%가 만성질환을 보유하고 있습니다.
  2. 만성질환 보유자 중 76.1%가 정신건강이 양호한 '긍정사례'입니다.
  3. 긍정사례는 위험군보다 평균 0.7일 더 많이 운동합니다.
  4. 만성질환과 가장 높은 상관관계: age (r=0.503)
  5. 고연령층의 만성질환 유병률은 최대 66.2%입니다.

💾 분석 결과 저장
✅ 긍정사례_샘플.csv 저장 완료 (100명)
✅ 건강프로필_요약통계.csv 저장 완료
✅ 상관계수_매트릭스.csv 저장 완료

🎉 분석 완료!

추천 후속 분석:
  1. 로지스틱 회귀분석으로 만성질환 위험요인 예측 모델 구축
  2. 군집분석(K-means)으로 건강행태 유형 분류
  3. 지역구별, 소득수준별 건강 격차 분석
  4. 긍정사례의 세부 생활습관 패턴 심층 분석

파일 다운로드는 좌측 폴더 아이콘에서 확인하세요!


In [3]:
# ====================================================================
# 건강행태 고급 분석 - 4대 후속 분석
# 1. 로지스틱 회귀분석 2. 군집분석 3. 격차분석 4. 긍정사례 심층분석
# ====================================================================

# 라이브러리 설치 및 임포트
print("📦 라이브러리 설치 중...")
!pip install plotly scikit-learn statsmodels -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
from scipy import stats
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

print("✅ 준비 완료!\n")

# ====================================================================
# 데이터 로드 및 전처리 (이전 분석과 동일)
# ====================================================================
print("📂 데이터 로딩...")
from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, encoding='cp949')
print(f"✅ 데이터 로드: {df.shape[0]:,}행 × {df.shape[1]}열\n")

# 변수 매핑
var_mapping = {
    'age': '만나이',
    'gender': '성별',
    'region': '행정기관코드',
    'hypertension': '고혈압 진단 경험 여부',
    'diabetes': '당뇨병 진단 경험 여부',
    'stress': '주관적 스트레스 수준',
    'depression': '우울감 경험 여부',
    'sleep_hours': '하루 평균 수면시간 중',
    'smoking': '일반담배 현재흡연 여부',
    'drinking_freq': '연간 음주 빈도',
    'high_exercise': '고강도활동 일수',
    'moderate_exercise': '중강도활동 일수',
    'walk_days': '10분걷기 일수',
    'breakfast': '주간 아침식사',
    'height': '키',
    'weight': '몸무게',
    'income': '가구소득 (월)금액',
    'education': '교육 수준',
    'subjective_health': '주관적 건강수준',
}

rename_dict = {v: k for k, v in var_mapping.items() if v in df.columns}
df_clean = df.rename(columns=rename_dict).copy()

# 결측값 처리
missing_codes = [8, 88, 888, 8888, 88888]
for col in df_clean.select_dtypes(include=[np.number]).columns:
    df_clean[col] = df_clean[col].replace(missing_codes, np.nan)

# 파생변수 생성
if 'height' in df_clean.columns and 'weight' in df_clean.columns:
    df_clean['bmi'] = df_clean['weight'] / ((df_clean['height'] / 100) ** 2)

if 'hypertension' in df_clean.columns:
    df_clean['has_hypertension'] = (df_clean['hypertension'] == 1).astype(int)
if 'diabetes' in df_clean.columns:
    df_clean['has_diabetes'] = (df_clean['diabetes'] == 1).astype(int)

df_clean['has_chronic'] = (
    (df_clean.get('has_hypertension', pd.Series([0])) == 1) |
    (df_clean.get('has_diabetes', pd.Series([0])) == 1)
).astype(int)

if 'stress' in df_clean.columns:
    df_clean['stress_level'] = 5 - df_clean['stress']
if 'depression' in df_clean.columns:
    df_clean['has_depression'] = (df_clean['depression'] == 1).astype(int)

df_clean['good_mental_health'] = (
    (df_clean.get('stress_level', 3) <= 2) &
    (df_clean.get('has_depression', pd.Series([0])) == 0)
).astype(int)

exercise_cols = ['high_exercise', 'moderate_exercise', 'walk_days']
available_exercise = [col for col in exercise_cols if col in df_clean.columns]
if available_exercise:
    df_clean['exercise_score'] = df_clean[available_exercise].fillna(0).sum(axis=1)

if 'age' in df_clean.columns:
    df_clean['age_group'] = pd.cut(df_clean['age'], bins=[0, 30, 40, 50, 60, 70, 100],
                                    labels=['20대이하', '30대', '40대', '50대', '60대', '70대이상'])

if 'gender' in df_clean.columns:
    df_clean['gender_label'] = df_clean['gender'].map({1: '남성', 2: '여성'})

# 건강 프로필
df_clean['health_profile'] = '기타'
df_clean.loc[df_clean['has_chronic'] == 0, 'health_profile'] = '만성질환없음'
df_clean.loc[(df_clean['has_chronic'] == 1) & (df_clean['good_mental_health'] == 1), 'health_profile'] = '긍정사례'
df_clean.loc[(df_clean['has_chronic'] == 1) & (df_clean['good_mental_health'] == 0), 'health_profile'] = '위험군'

# 소득 구간 (월소득)
if 'income' in df_clean.columns:
    df_clean['income_group'] = pd.cut(
        df_clean['income'],
        bins=[-1, 200, 400, 600, 10000],
        labels=['200만원이하', '200-400만원', '400-600만원', '600만원이상']
    )

# 지역구 (서울시 자치구)
if 'region' in df_clean.columns:
    region_map = {
        11110: '종로구', 11140: '중구', 11170: '용산구', 11200: '성동구',
        11215: '광진구', 11230: '동대문구', 11260: '중랑구', 11290: '성북구',
        11305: '강북구', 11320: '도봉구', 11350: '노원구', 11380: '은평구',
        11410: '서대문구', 11440: '마포구', 11470: '양천구', 11500: '강서구',
        11530: '구로구', 11545: '금천구', 11560: '영등포구', 11590: '동작구',
        11620: '관악구', 11650: '서초구', 11680: '강남구', 11710: '송파구', 11740: '강동구'
    }
    df_clean['region_name'] = df_clean['region'].map(region_map)

print(f"✅ 전처리 완료: {df_clean.shape[0]:,}행\n")

# ====================================================================
# 분석 1: 로지스틱 회귀분석 - 만성질환 위험요인 예측
# ====================================================================
print("="*70)
print("📊 분석 1: 로지스틱 회귀분석 - 만성질환 위험요인 예측 모델")
print("="*70)

# 독립변수 선택
predictor_vars = ['age', 'gender', 'bmi', 'exercise_score', 'stress_level',
                  'smoking', 'drinking_freq', 'breakfast', 'sleep_hours']
available_predictors = [v for v in predictor_vars if v in df_clean.columns]

# 결측값 제거
model_data = df_clean[available_predictors + ['has_chronic']].dropna()
print(f"\n분석 대상: {len(model_data):,}명 (결측값 제외)")

if len(model_data) > 100:
    X = model_data[available_predictors]
    y = model_data['has_chronic']

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # 표준화
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 모델 학습
    log_model = LogisticRegression(random_state=42, max_iter=1000)
    log_model.fit(X_train_scaled, y_train)

    # 예측 및 평가
    y_pred = log_model.predict(X_test_scaled)
    y_pred_proba = log_model.predict_proba(X_test_scaled)[:, 1]

    from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)

    print(f"\n모델 성능:")
    print(f"  정확도(Accuracy): {accuracy:.3f}")
    print(f"  AUC Score: {auc_score:.3f}")

    # 회귀계수 해석
    coefficients = pd.DataFrame({
        '변수': available_predictors,
        '회귀계수': log_model.coef_[0],
        'Odds Ratio': np.exp(log_model.coef_[0])
    }).sort_values('회귀계수', ascending=False)

    print(f"\n회귀계수 (Odds Ratio):")
    print(coefficients.round(3))

    print(f"\n💡 해석:")
    top_risk = coefficients.iloc[0]
    print(f"  - 가장 큰 위험요인: {top_risk['변수']} (OR={top_risk['Odds Ratio']:.2f})")
    top_protect = coefficients.iloc[-1]
    print(f"  - 가장 큰 보호요인: {top_protect['변수']} (OR={top_protect['Odds Ratio']:.2f})")

    # ROC Curve 시각화
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f'ROC Curve (AUC={auc_score:.3f})',
                             line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], name='Random',
                             line=dict(color='red', dash='dash')))
    fig.update_layout(
        title='ROC Curve - 만성질환 예측 모델',
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        width=700, height=500
    )
    fig.show()

    # 변수 중요도 시각화
    fig2 = go.Figure(go.Bar(
        x=coefficients['회귀계수'],
        y=coefficients['변수'],
        orientation='h',
        marker=dict(color=coefficients['회귀계수'],
                   colorscale='RdBu_r', cmid=0)
    ))
    fig2.update_layout(
        title='만성질환 위험요인 회귀계수',
        xaxis_title='회귀계수 (양수=위험증가, 음수=위험감소)',
        yaxis_title='',
        width=700, height=500
    )
    fig2.show()

    # 결과 저장
    coefficients.to_csv('로지스틱회귀_계수.csv', index=False, encoding='utf-8-sig')
    print("\n✅ 로지스틱회귀_계수.csv 저장 완료")
else:
    print("⚠️ 데이터가 부족하여 모델 학습을 진행할 수 없습니다.")

# ====================================================================
# 분석 2: 군집분석 (K-means) - 건강행태 유형별 그룹 분류
# ====================================================================
print("\n" + "="*70)
print("📊 분석 2: 군집분석 (K-means) - 건강행태 유형 분류")
print("="*70)

# 군집분석용 변수 선택
cluster_vars = ['exercise_score', 'stress_level', 'bmi', 'sleep_hours', 'breakfast']
available_cluster = [v for v in cluster_vars if v in df_clean.columns]

cluster_data = df_clean[available_cluster].dropna()
print(f"\n분석 대상: {len(cluster_data):,}명")

if len(cluster_data) > 100:
    # 표준화
    scaler_cluster = StandardScaler()
    cluster_scaled = scaler_cluster.fit_transform(cluster_data)

    # 최적 군집 수 찾기 (Elbow Method & Silhouette)
    print("\n최적 군집 수 탐색 중...")
    inertias = []
    silhouettes = []
    K_range = range(2, 8)

    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(cluster_scaled)
        inertias.append(kmeans.inertia_)
        silhouettes.append(silhouette_score(cluster_scaled, kmeans.labels_))

    # Elbow & Silhouette 시각화
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Elbow Method', 'Silhouette Score'))

    fig.add_trace(go.Scatter(x=list(K_range), y=inertias, mode='lines+markers', name='Inertia'),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=list(K_range), y=silhouettes, mode='lines+markers', name='Silhouette',
                            marker=dict(color='orange')),
                  row=1, col=2)

    fig.update_xaxes(title_text="군집 수 (K)", row=1, col=1)
    fig.update_xaxes(title_text="군집 수 (K)", row=1, col=2)
    fig.update_layout(title="최적 군집 수 결정", height=400, width=900)
    fig.show()

    # 최적 군집수로 최종 모델링 (Silhouette 최대값 사용)
    optimal_k = list(K_range)[np.argmax(silhouettes)]
    print(f"\n최적 군집 수: {optimal_k}개 (Silhouette Score: {max(silhouettes):.3f})")

    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    df_clean.loc[cluster_data.index, 'cluster'] = kmeans_final.fit_predict(cluster_scaled)

    # 군집별 특성 분석
    print(f"\n군집별 인원 분포:")
    cluster_counts = df_clean['cluster'].value_counts().sort_index()
    for cluster_id, count in cluster_counts.items():
        print(f"  군집 {int(cluster_id)}: {count:,}명 ({count/len(cluster_data)*100:.1f}%)")

    # 군집별 평균 특성
    cluster_profile = df_clean.groupby('cluster')[available_cluster].mean().round(2)
    print(f"\n군집별 건강행태 프로필:")
    print(cluster_profile)

    # 군집 명명
    cluster_names = {}
    for i in range(optimal_k):
        profile = cluster_profile.loc[i]
        if 'exercise_score' in profile and profile['exercise_score'] > cluster_profile['exercise_score'].median():
            if 'stress_level' in profile and profile['stress_level'] < cluster_profile['stress_level'].median():
                cluster_names[i] = '활동적-안정형'
            else:
                cluster_names[i] = '활동적-스트레스형'
        else:
            if 'stress_level' in profile and profile['stress_level'] < cluster_profile['stress_level'].median():
                cluster_names[i] = '비활동-안정형'
            else:
                cluster_names[i] = '비활동-스트레스형'

    df_clean['cluster_name'] = df_clean['cluster'].map(cluster_names)

    print(f"\n군집 명명:")
    for cluster_id, name in cluster_names.items():
        print(f"  군집 {int(cluster_id)}: {name}")

    # 군집별 만성질환 유병률
    cluster_chronic = df_clean.groupby('cluster_name')['has_chronic'].agg(['mean', 'count'])
    cluster_chronic.columns = ['만성질환_유병률', '인원수']
    cluster_chronic['만성질환_유병률'] = (cluster_chronic['만성질환_유병률'] * 100).round(1)
    print(f"\n군집별 만성질환 유병률(%):")
    print(cluster_chronic)

    # 3D 시각화 (처음 3개 변수)
    if len(available_cluster) >= 3:
        fig = px.scatter_3d(
            df_clean.dropna(subset=['cluster_name']),
            x=available_cluster[0],
            y=available_cluster[1],
            z=available_cluster[2],
            color='cluster_name',
            title='건강행태 군집 3D 시각화',
            labels={available_cluster[0]: available_cluster[0],
                   available_cluster[1]: available_cluster[1],
                   available_cluster[2]: available_cluster[2]},
            height=600
        )
        fig.show()

    # 결과 저장
    cluster_profile.to_csv('군집별_건강행태_프로필.csv', encoding='utf-8-sig')
    print("\n✅ 군집별_건강행태_프로필.csv 저장 완료")
else:
    print("⚠️ 데이터가 부족합니다.")

# ====================================================================
# 분석 3: 지역구별/소득별 건강 격차 분석
# ====================================================================
print("\n" + "="*70)
print("📊 분석 3: 지역구별/소득별 건강 격차 분석")
print("="*70)

# 3-1. 소득별 격차
if 'income_group' in df_clean.columns:
    print("\n[소득별 건강 격차]")

    income_analysis = df_clean.groupby('income_group').agg({
        'has_chronic': 'mean',
        'good_mental_health': 'mean',
        'exercise_score': 'mean',
        'bmi': 'mean'
    }).round(3) * 100  # 비율을 퍼센트로

    income_analysis.columns = ['만성질환_유병률(%)', '정신건강_양호(%)', '평균운동점수(%)', '평균BMI(%)']
    # BMI는 평균값이므로 다시 원래대로
    income_analysis['평균BMI(%)'] = df_clean.groupby('income_group')['bmi'].mean().round(2)

    print(income_analysis)

    # 통계적 유의성 검정 (Kruskal-Wallis test)
    print("\n통계적 유의성 검정 (Kruskal-Wallis):")
    for var in ['has_chronic', 'exercise_score']:
        if var in df_clean.columns:
            groups = [group[var].dropna() for name, group in df_clean.groupby('income_group')]
            if len(groups) > 1 and all(len(g) > 0 for g in groups):
                h_stat, p_value = stats.kruskal(*groups)
                sig = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                print(f"  {var}: p={p_value:.4f} {sig}")

    # 시각화
    fig = go.Figure()
    for col in ['만성질환_유병률(%)', '정신건강_양호(%)']:
        if col in income_analysis.columns:
            fig.add_trace(go.Bar(
                x=income_analysis.index,
                y=income_analysis[col],
                name=col
            ))

    fig.update_layout(
        title='소득 수준별 건강 격차',
        xaxis_title='소득 구간',
        yaxis_title='비율 (%)',
        barmode='group',
        height=500
    )
    fig.show()
else:
    print("\n⚠️ 소득 변수가 없습니다.")

# 3-2. 지역구별 격차
if 'region_name' in df_clean.columns:
    print("\n[지역구별 건강 격차]")

    region_analysis = df_clean.groupby('region_name').agg({
        'has_chronic': ['mean', 'count'],
        'good_mental_health': 'mean',
        'exercise_score': 'mean'
    }).round(3)

    region_analysis.columns = ['만성질환_유병률', '인원수', '정신건강_양호율', '평균운동점수']
    region_analysis['만성질환_유병률'] = (region_analysis['만성질환_유병률'] * 100).round(1)
    region_analysis['정신건강_양호율'] = (region_analysis['정신건강_양호율'] * 100).round(1)
    region_analysis = region_analysis.sort_values('만성질환_유병률', ascending=False)

    print("\n상위 10개 자치구 (만성질환 유병률 기준):")
    print(region_analysis.head(10))

    print("\n하위 10개 자치구 (만성질환 유병률 기준):")
    print(region_analysis.tail(10))

    # 지역 격차 시각화
    fig = px.bar(
        region_analysis.reset_index().head(15),
        x='region_name',
        y='만성질환_유병률',
        color='만성질환_유병률',
        title='자치구별 만성질환 유병률 (상위 15개)',
        labels={'region_name': '자치구', '만성질환_유병률': '유병률 (%)'},
        color_continuous_scale='Reds',
        height=500
    )
    fig.show()

    # 결과 저장
    region_analysis.to_csv('지역구별_건강격차.csv', encoding='utf-8-sig')
    print("\n✅ 지역구별_건강격차.csv 저장 완료")
else:
    print("\n⚠️ 지역 변수가 없습니다.")

# 격차 지수 계산
if 'income_group' in df_clean.columns:
    highest_income = df_clean[df_clean['income_group'] == '600만원이상']['has_chronic'].mean()
    lowest_income = df_clean[df_clean['income_group'] == '200만원이하']['has_chronic'].mean()
    income_gap = (lowest_income - highest_income) / highest_income * 100
    print(f"\n💡 소득 격차 지수: 저소득층이 고소득층보다 만성질환 유병률이 {abs(income_gap):.1f}% {'높음' if income_gap > 0 else '낮음'}")

# ====================================================================
# 분석 4: 긍정사례 심층 분석 - 구체적 생활습관 패턴
# ====================================================================
print("\n" + "="*70)
print("📊 분석 4: 긍정사례 심층 분석 - 구체적인 생활습관 패턴")
print("="*70)

positive_group = df_clean[df_clean['health_profile'] == '긍정사례']
risk_group = df_clean[df_clean['health_profile'] == '위험군']
healthy_group = df_clean[df_clean['health_profile'] == '만성질환없음']

print(f"\n그룹별 인원:")
print(f"  긍정사례: {len(positive_group):,}명")
print(f"  위험군: {len(risk_group):,}명")
print(f"  만성질환없음: {len(healthy_group):,}명")

if len(positive_group) > 0:
    # 4-1. 생활습관 상세 비교
    lifestyle_vars = ['exercise_score', 'high_exercise', 'moderate_exercise', 'walk_days',
                     'breakfast', 'sleep_hours', 'smoking', 'drinking_freq', 'bmi']
    available_lifestyle = [v for v in lifestyle_vars if v in df_clean.columns]

    lifestyle_comparison = pd.DataFrame({
        '긍정사례': positive_group[available_lifestyle].mean(),
        '위험군': risk_group[available_lifestyle].mean(),
        '건강군': healthy_group[available_lifestyle].mean(),
        '긍정vs위험_차이': positive_group[available_lifestyle].mean() - risk_group[available_lifestyle].mean()
    }).round(2)

    print(f"\n생활습관 상세 비교:")
    print(lifestyle_comparison)

    # 통계적 유의성
    print(f"\n긍정사례 vs 위험군 통계 검정 (t-test):")
    for var in available_lifestyle:
        pos_data = positive_group[var].dropna()
        risk_data = risk_group[var].dropna()
        if len(pos_data) > 0 and len(risk_data) > 0:
            t_stat, p_value = stats.ttest_ind(pos_data, risk_data)
            sig = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
            print(f"  {var}: p={p_value:.4f} {sig}")

    # 4-2. 긍정사례의 특징적 패턴 발견
    print(f"\n긍정사례의 특징적 패턴:")

    # 운동 실천률
    if 'exercise_score' in df_clean.columns:
        high_exercise_positive = (positive_group['exercise_score'] >= 7).mean() * 100
        high_exercise_risk = (risk_group['exercise_score'] >= 7).mean() * 100
        print(f"  1. 주 7일 이상 운동: 긍정사례 {high_exercise_positive:.1f}% vs 위험군 {high_exercise_risk:.1f}%")

    # 아침식사 실천률
    if 'breakfast' in df_clean.columns:
        breakfast_positive = (positive_group['breakfast'] >= 5).mean() * 100
        breakfast_risk = (risk_group['breakfast'] >= 5).mean() * 100
        print(f"  2. 주 5일 이상 아침식사: 긍정사례 {breakfast_positive:.1f}% vs 위험군 {breakfast_risk:.1f}%")

    # 적정 수면
    if 'sleep_hours' in df_clean.columns:
        good_sleep_positive = ((positive_group['sleep_hours'] >= 6) & (positive_group['sleep_hours'] <= 8)).mean() * 100
        good_sleep_risk = ((risk_group['sleep_hours'] >= 6) & (risk_group['sleep_hours'] <= 8)).mean() * 100
        print(f"  3. 적정 수면(6-8시간): 긍정사례 {good_sleep_positive:.1f}% vs 위험군 {good_sleep_risk:.1f}%")

    # 비흡연율
    if 'smoking' in df_clean.columns:
        nonsmoking_positive = (positive_group['smoking'] == 3).mean() * 100
        nonsmoking_risk = (risk_group['smoking'] == 3).mean() * 100
        print(f"  4. 비흡연율: 긍정사례 {nonsmoking_positive:.1f}% vs 위험군 {nonsmoking_risk:.1f}%")

    # 정상 BMI
    if 'bmi' in df_clean.columns:
        normal_bmi_positive = ((positive_group['bmi'] >= 18.5) & (positive_group['bmi'] < 25)).mean() * 100
        normal_bmi_risk = ((risk_group['bmi'] >= 18.5) & (risk_group['bmi'] < 25)).mean() * 100
        print(f"  5. 정상 BMI(18.5-25): 긍정사례 {normal_bmi_positive:.1f}% vs 위험군 {normal_bmi_risk:.1f}%")

    # 4-3. 긍정사례 성공 요인 스코어링
    print(f"\n긍정사례 성공 요인 분석:")

    success_factors = []
    if 'exercise_score' in lifestyle_comparison.index:
        exercise_effect = lifestyle_comparison.loc['exercise_score', '긍정vs위험_차이']
        success_factors.append(('운동', exercise_effect))
    if 'breakfast' in lifestyle_comparison.index:
        breakfast_effect = lifestyle_comparison.loc['breakfast', '긍정vs위험_차이']
        success_factors.append(('아침식사', breakfast_effect))
    if 'sleep_hours' in lifestyle_comparison.index:
        sleep_effect = lifestyle_comparison.loc['sleep_hours', '긍정vs위험_차이']
        success_factors.append(('수면시간', sleep_effect))

    success_factors.sort(key=lambda x: abs(x[1]), reverse=True)

    print("  핵심 성공 요인 (차이가 큰 순서):")
    for i, (factor, diff) in enumerate(success_factors[:5], 1):
        direction = "더 많음" if diff > 0 else "더 적음"
        print(f"    {i}. {factor}: {abs(diff):.2f} {direction}")

    # 4-4. 종합 시각화 - 레이더 차트
    if len(available_lifestyle) >= 5:
        # 상위 5개 변수로 레이더 차트
        radar_vars = available_lifestyle[:5]

        # 정규화 (0-100 스케일)
        radar_data = {}
        for var in radar_vars:
            max_val = df_clean[var].max()
            min_val = df_clean[var].min()
            if max_val > min_val:
                radar_data[var] = {
                    '긍정사례': (positive_group[var].mean() - min_val) / (max_val - min_val) * 100,
                    '위험군': (risk_group[var].mean() - min_val) / (max_val - min_val) * 100,
                    '건강군': (healthy_group[var].mean() - min_val) / (max_val - min_val) * 100
                }

        fig = go.Figure()

        for group_name in ['긍정사례', '위험군', '건강군']:
            values = [radar_data[var][group_name] for var in radar_vars]
            fig.add_trace(go.Scatterpolar(
                r=values + [values[0]],  # 첫 값을 마지막에 추가하여 폐곡선
                theta=radar_vars + [radar_vars[0]],
                fill='toself',
                name=group_name
            ))

        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
            title='건강 프로필별 생활습관 패턴 (레이더 차트)',
            height=600
        )
        fig.show()

    # 4-5. 긍정사례 세부 유형 분석
    print(f"\n긍정사례 세부 유형 분석:")

    # 운동형 vs 비운동형
    if 'exercise_score' in positive_group.columns:
        exercise_median = positive_group['exercise_score'].median()
        high_exercise_positive = positive_group[positive_group['exercise_score'] >= exercise_median]
        low_exercise_positive = positive_group[positive_group['exercise_score'] < exercise_median]

        print(f"  고운동형 긍정사례: {len(high_exercise_positive)}명")
        print(f"  저운동형 긍정사례: {len(low_exercise_positive)}명")

        if len(low_exercise_positive) > 0:
            print(f"\n  💡 저운동형 긍정사례의 대체 건강습관:")
            for var in ['breakfast', 'sleep_hours', 'bmi']:
                if var in low_exercise_positive.columns:
                    avg_val = low_exercise_positive[var].mean()
                    overall_avg = positive_group[var].mean()
                    print(f"    - {var}: {avg_val:.2f} (전체 긍정사례 평균: {overall_avg:.2f})")

    # 4-6. 생활습관 조합 패턴
    print(f"\n긍정사례의 생활습관 조합 패턴:")

    positive_patterns = positive_group.copy()

    # 건강습관 실천 개수
    health_practices = []
    if 'exercise_score' in positive_patterns.columns:
        health_practices.append((positive_patterns['exercise_score'] >= 7).astype(int))
    if 'breakfast' in positive_patterns.columns:
        health_practices.append((positive_patterns['breakfast'] >= 5).astype(int))
    if 'sleep_hours' in positive_patterns.columns:
        health_practices.append(((positive_patterns['sleep_hours'] >= 6) &
                                 (positive_patterns['sleep_hours'] <= 8)).astype(int))
    if 'smoking' in positive_patterns.columns:
        health_practices.append((positive_patterns['smoking'] == 3).astype(int))

    if health_practices:
        positive_patterns['health_practice_count'] = pd.concat(health_practices, axis=1).sum(axis=1)

        practice_dist = positive_patterns['health_practice_count'].value_counts().sort_index()
        print(f"  건강습관 실천 개수 분포:")
        for count, freq in practice_dist.items():
            pct = freq / len(positive_patterns) * 100
            print(f"    {int(count)}개 실천: {freq}명 ({pct:.1f}%)")

    # 4-7. 긍정사례 인구학적 특성
    print(f"\n긍정사례 인구학적 특성:")

    if 'age' in positive_group.columns:
        avg_age = positive_group['age'].mean()
        print(f"  평균 연령: {avg_age:.1f}세")

    if 'gender_label' in positive_group.columns:
        gender_dist = positive_group['gender_label'].value_counts()
        print(f"  성별 분포:")
        for gender, count in gender_dist.items():
            pct = count / len(positive_group) * 100
            print(f"    {gender}: {count}명 ({pct:.1f}%)")

    if 'income_group' in positive_group.columns:
        income_dist = positive_group['income_group'].value_counts().sort_index()
        print(f"  소득 분포:")
        for income, count in income_dist.items():
            pct = count / len(positive_group) * 100
            print(f"    {income}: {count}명 ({pct:.1f}%)")

    if 'education' in positive_group.columns:
        edu_avg = positive_group['education'].mean()
        print(f"  평균 교육수준: {edu_avg:.2f}")

    # 결과 저장
    lifestyle_comparison.to_csv('긍정사례_생활습관_상세비교.csv', encoding='utf-8-sig')
    print(f"\n✅ 긍정사례_생활습관_상세비교.csv 저장 완료")

    # 긍정사례 프로필 저장
    positive_profile_cols = ['age', 'gender_label', 'income_group', 'education',
                            'exercise_score', 'breakfast', 'sleep_hours', 'bmi']
    available_profile = [col for col in positive_profile_cols if col in positive_group.columns]
    if available_profile:
        positive_sample = positive_group[available_profile].sample(min(200, len(positive_group)))
        positive_sample.to_csv('긍정사례_상세프로필_샘플.csv', index=False, encoding='utf-8-sig')
        print(f"✅ 긍정사례_상세프로필_샘플.csv 저장 완료")

else:
    print("⚠️ 긍정사례 데이터가 부족합니다.")

# ====================================================================
# 종합 요약 및 정책 제언
# ====================================================================
print("\n" + "="*70)
print("📋 종합 요약 및 정책 제언")
print("="*70)

print("\n[주요 발견사항]")

findings = []

# 1. 로지스틱 회귀
if 'log_model' in locals():
    top_risk_factor = coefficients.iloc[0]
    findings.append(f"1. 만성질환의 가장 큰 위험요인: {top_risk_factor['변수']} (OR={top_risk_factor['Odds Ratio']:.2f})")

# 2. 군집분석
if 'optimal_k' in locals():
    high_risk_cluster = cluster_chronic.sort_values('만성질환_유병률', ascending=False).index[0]
    high_risk_rate = cluster_chronic.loc[high_risk_cluster, '만성질환_유병률']
    findings.append(f"2. 가장 고위험 건강행태 유형: {high_risk_cluster} (유병률 {high_risk_rate}%)")

# 3. 격차분석
if 'income_gap' in locals():
    findings.append(f"3. 소득 격차: 저소득층이 고소득층보다 만성질환 위험 {abs(income_gap):.1f}% 높음")

# 4. 긍정사례
if len(positive_group) > 0 and success_factors:
    top_success = success_factors[0]
    findings.append(f"4. 긍정사례의 핵심 성공요인: {top_success[0]} (차이 {abs(top_success[1]):.2f})")

for finding in findings:
    print(f"  {finding}")

print("\n[정책 제언]")

recommendations = [
    "1. 맞춤형 건강관리: 군집별로 차별화된 건강증진 프로그램 개발",
    "2. 취약계층 집중 지원: 저소득층 대상 무료 건강검진 및 운동 프로그램",
    "3. 지역 기반 접근: 유병률 높은 자치구에 보건소 건강증진센터 확대",
    "4. 긍정사례 모델링: 만성질환자의 정신건강 관리 프로그램 강화",
    "5. 예방적 개입: 고위험 건강행태 그룹 조기 발견 및 개입 시스템 구축"
]

for rec in recommendations:
    print(f"  {rec}")

# ====================================================================
# 전체 결과 종합 리포트 생성
# ====================================================================
print("\n" + "="*70)
print("📊 최종 결과 요약")
print("="*70)

summary_report = {
    '분석항목': [
        '전체 대상자',
        '만성질환 유병률',
        '긍정사례 비율',
        '예측모델 정확도',
        '건강행태 군집수',
        '소득 격차(배수)'
    ],
    '결과': [
        f"{len(df_clean):,}명",
        f"{df_clean['has_chronic'].mean()*100:.1f}%",
        f"{len(positive_group)/len(df_clean)*100:.1f}%",
        f"{accuracy:.3f}" if 'accuracy' in locals() else 'N/A',
        f"{optimal_k}개" if 'optimal_k' in locals() else 'N/A',
        f"{abs(income_gap/100):.2f}배" if 'income_gap' in locals() else 'N/A'
    ]
}

summary_df = pd.DataFrame(summary_report)
print(summary_df.to_string(index=False))

# 종합 리포트 저장
summary_df.to_csv('종합분석_결과요약.csv', index=False, encoding='utf-8-sig')
print("\n✅ 종합분석_결과요약.csv 저장 완료")

# ====================================================================
# 인터랙티브 종합 대시보드
# ====================================================================
print("\n📊 종합 대시보드 생성 중...")

fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        '예측모델 변수 중요도',
        '건강행태 군집 분포',
        '소득별 건강 격차',
        '지역별 유병률 (상위10)',
        '긍정사례 vs 위험군 비교',
        '생활습관 실천률 비교'
    ),
    specs=[
        [{'type': 'bar'}, {'type': 'pie'}],
        [{'type': 'bar'}, {'type': 'bar'}],
        [{'type': 'bar'}, {'type': 'bar'}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.15
)

# 차트 1: 회귀계수
if 'coefficients' in locals():
    top_coef = coefficients.head(8)
    fig.add_trace(
        go.Bar(x=top_coef['회귀계수'], y=top_coef['변수'], orientation='h',
               marker_color='steelblue'),
        row=1, col=1
    )

# 차트 2: 군집 분포
if 'cluster_counts' in locals():
    fig.add_trace(
        go.Pie(labels=[f'군집{int(i)}' for i in cluster_counts.index],
               values=cluster_counts.values),
        row=1, col=2
    )

# 차트 3: 소득별 격차
if 'income_analysis' in locals():
    fig.add_trace(
        go.Bar(x=income_analysis.index, y=income_analysis['만성질환_유병률(%)'],
               marker_color='coral'),
        row=2, col=1
    )

# 차트 4: 지역별 유병률
if 'region_analysis' in locals():
    top_regions = region_analysis.head(10)
    fig.add_trace(
        go.Bar(x=top_regions.index, y=top_regions['만성질환_유병률'],
               marker_color='indianred'),
        row=2, col=2
    )

# 차트 5: 긍정사례 vs 위험군
if 'lifestyle_comparison' in locals() and len(lifestyle_comparison) > 0:
    comparison_vars = lifestyle_comparison.index[:5]
    fig.add_trace(
        go.Bar(x=comparison_vars, y=lifestyle_comparison.loc[comparison_vars, '긍정사례'],
               name='긍정사례', marker_color='green'),
        row=3, col=1
    )
    fig.add_trace(
        go.Bar(x=comparison_vars, y=lifestyle_comparison.loc[comparison_vars, '위험군'],
               name='위험군', marker_color='red'),
        row=3, col=1
    )

# 차트 6: 생활습관 실천률
if len(positive_group) > 0 and len(risk_group) > 0:
    practices = ['운동7일이상', '아침식사5일', '적정수면', '비흡연']
    pos_rates = []
    risk_rates = []

    if 'exercise_score' in positive_group.columns:
        pos_rates.append((positive_group['exercise_score'] >= 7).mean() * 100)
        risk_rates.append((risk_group['exercise_score'] >= 7).mean() * 100)
    if 'breakfast' in positive_group.columns:
        pos_rates.append((positive_group['breakfast'] >= 5).mean() * 100)
        risk_rates.append((risk_group['breakfast'] >= 5).mean() * 100)
    if 'sleep_hours' in positive_group.columns:
        pos_rates.append(((positive_group['sleep_hours'] >= 6) &
                         (positive_group['sleep_hours'] <= 8)).mean() * 100)
        risk_rates.append(((risk_group['sleep_hours'] >= 6) &
                          (risk_group['sleep_hours'] <= 8)).mean() * 100)
    if 'smoking' in positive_group.columns:
        pos_rates.append((positive_group['smoking'] == 3).mean() * 100)
        risk_rates.append((risk_group['smoking'] == 3).mean() * 100)

    practices = practices[:len(pos_rates)]

    fig.add_trace(
        go.Bar(x=practices, y=pos_rates, name='긍정사례', marker_color='lightgreen'),
        row=3, col=2
    )
    fig.add_trace(
        go.Bar(x=practices, y=risk_rates, name='위험군', marker_color='lightcoral'),
        row=3, col=2
    )

fig.update_layout(
    title_text="건강행태-만성질환 종합 분석 대시보드",
    height=1400,
    showlegend=True
)

fig.show()

print("\n" + "="*70)
print("🎉 4대 후속 분석 완료!")
print("="*70)
print("\n생성된 파일:")
print("  1. 로지스틱회귀_계수.csv")
print("  2. 군집별_건강행태_프로필.csv")
print("  3. 지역구별_건강격차.csv")
print("  4. 긍정사례_생활습관_상세비교.csv")
print("  5. 긍정사례_상세프로필_샘플.csv")
print("  6. 종합분석_결과요약.csv")
print("\n좌측 폴더 아이콘에서 다운로드 가능합니다!")
print("\n다음 단계: 시각화 고도화, 특정 변수 집중 분석 등을 요청하세요! 💪")

📦 라이브러리 설치 중...
✅ 준비 완료!

📂 데이터 로딩...


Saving 서울2024_변수가공.csv to 서울2024_변수가공 (2).csv
✅ 데이터 로드: 22,906행 × 66열

✅ 전처리 완료: 22,906행

📊 분석 1: 로지스틱 회귀분석 - 만성질환 위험요인 예측 모델

분석 대상: 19,272명 (결측값 제외)

모델 성능:
  정확도(Accuracy): 0.758
  AUC Score: 0.823

회귀계수 (Odds Ratio):
               변수   회귀계수  Odds Ratio
0             age  1.397       4.044
4    stress_level  0.089       1.093
2             bmi  0.056       1.057
5       breakfast -0.036       0.965
3  exercise_score -0.085       0.919
6     sleep_hours -0.098       0.906
1          gender -0.246       0.782

💡 해석:
  - 가장 큰 위험요인: age (OR=4.04)
  - 가장 큰 보호요인: gender (OR=0.78)



✅ 로지스틱회귀_계수.csv 저장 완료

📊 분석 2: 군집분석 (K-means) - 건강행태 유형 분류

분석 대상: 19,322명

최적 군집 수 탐색 중...



최적 군집 수: 2개 (Silhouette Score: 0.891)

군집별 인원 분포:
  군집 0: 19,283명 (99.8%)
  군집 1: 39명 (0.2%)

군집별 건강행태 프로필:
         exercise_score  stress_level       bmi  sleep_hours  breakfast
cluster                                                                
0.0                7.72          2.04     23.54         6.25       1.98
1.0                6.77          2.51  34105.52         6.28       2.13

군집 명명:
  군집 0: 활동적-안정형
  군집 1: 비활동-스트레스형

군집별 만성질환 유병률(%):
              만성질환_유병률    인원수
cluster_name                 
비활동-스트레스형         30.8     39
활동적-안정형           31.0  19283



✅ 군집별_건강행태_프로필.csv 저장 완료

📊 분석 3: 지역구별/소득별 건강 격차 분석

⚠️ 소득 변수가 없습니다.

[지역구별 건강 격차]

상위 10개 자치구 (만성질환 유병률 기준):
             만성질환_유병률  인원수  정신건강_양호율  평균운동점수
region_name                                 
강북구              41.2  915      71.5   5.952
도봉구              37.7  915      73.9   7.611
금천구              36.1  915      70.7   7.634
성북구              35.3  915      74.0   7.930
구로구              35.2  915      74.3   7.152
은평구              34.6  915      76.5   7.318
중랑구              33.9  915      74.2   7.510
동대문구             31.7  916      81.0   6.691
성동구              31.7  915      75.2   8.078
관악구              31.4  915      71.3   7.452

하위 10개 자치구 (만성질환 유병률 기준):
             만성질환_유병률  인원수  정신건강_양호율  평균운동점수
region_name                                 
서대문구             29.7  915      69.9   7.172
노원구              29.6  916      74.6   7.591
마포구              29.3  916      70.7   7.787
강동구              29.0  920      75.7   8.389
용산구              28.7  923      86.1   7.696
중구     


✅ 지역구별_건강격차.csv 저장 완료

📊 분석 4: 긍정사례 심층 분석 - 구체적인 생활습관 패턴

그룹별 인원:
  긍정사례: 5,398명
  위험군: 1,697명
  만성질환없음: 15,811명

생활습관 상세 비교:
                    긍정사례     위험군    건강군  긍정vs위험_차이
exercise_score      7.08    6.37   8.02       0.71
high_exercise       0.64    0.58   1.13       0.06
moderate_exercise   1.15    0.92   1.40       0.23
walk_days           5.29    4.87   5.49       0.42
breakfast           1.43    1.74   2.19      -0.32
sleep_hours         6.20    5.93   6.31       0.27
bmi                83.09  161.66  88.57     -78.56

긍정사례 vs 위험군 통계 검정 (t-test):
  exercise_score: p=0.0000 ***
  high_exercise: p=0.1552 ns
  moderate_exercise: p=0.0000 ***
  walk_days: p=0.0000 ***
  breakfast: p=0.0000 ***
  sleep_hours: p=0.0000 ***
  bmi: p=0.0977 ns

긍정사례의 특징적 패턴:
  1. 주 7일 이상 운동: 긍정사례 62.3% vs 위험군 54.1%
  2. 주 5일 이상 아침식사: 긍정사례 0.0% vs 위험군 0.0%
  3. 적정 수면(6-8시간): 긍정사례 59.4% vs 위험군 49.0%
  5. 정상 BMI(18.5-25): 긍정사례 59.7% vs 위험군 52.9%

긍정사례 성공 요인 분석:
  핵심 성공 요인 (차이가 큰 순서):
    1. 운동: 0.71 더 


긍정사례 세부 유형 분석:
  고운동형 긍정사례: 3363명
  저운동형 긍정사례: 2035명

  💡 저운동형 긍정사례의 대체 건강습관:
    - breakfast: 1.49 (전체 긍정사례 평균: 1.43)
    - sleep_hours: 6.23 (전체 긍정사례 평균: 6.20)
    - bmi: 99.90 (전체 긍정사례 평균: 83.09)

긍정사례의 생활습관 조합 패턴:
  건강습관 실천 개수 분포:
    0개 실천: 879명 (16.3%)
    1개 실천: 2467명 (45.7%)
    2개 실천: 2052명 (38.0%)

긍정사례 인구학적 특성:
  평균 연령: 67.4세
  성별 분포:
    남성: 2713명 (50.3%)
    여성: 2685명 (49.7%)
  평균 교육수준: 4.80

✅ 긍정사례_생활습관_상세비교.csv 저장 완료
✅ 긍정사례_상세프로필_샘플.csv 저장 완료

📋 종합 요약 및 정책 제언

[주요 발견사항]
  1. 만성질환의 가장 큰 위험요인: age (OR=4.04)
  2. 가장 고위험 건강행태 유형: 활동적-안정형 (유병률 31.0%)
  4. 긍정사례의 핵심 성공요인: 운동 (차이 0.71)

[정책 제언]
  1. 맞춤형 건강관리: 군집별로 차별화된 건강증진 프로그램 개발
  2. 취약계층 집중 지원: 저소득층 대상 무료 건강검진 및 운동 프로그램
  3. 지역 기반 접근: 유병률 높은 자치구에 보건소 건강증진센터 확대
  4. 긍정사례 모델링: 만성질환자의 정신건강 관리 프로그램 강화
  5. 예방적 개입: 고위험 건강행태 그룹 조기 발견 및 개입 시스템 구축

📊 최종 결과 요약
     분석항목      결과
   전체 대상자 22,906명
 만성질환 유병률   31.0%
  긍정사례 비율   23.6%
 예측모델 정확도   0.758
 건강행태 군집수      2개
소득 격차(배수)     N/A

✅ 종합분석_결과요약.csv 저장 완료

📊 종합 대시보드 생성 중...



🎉 4대 후속 분석 완료!

생성된 파일:
  1. 로지스틱회귀_계수.csv
  2. 군집별_건강행태_프로필.csv
  3. 지역구별_건강격차.csv
  4. 긍정사례_생활습관_상세비교.csv
  5. 긍정사례_상세프로필_샘플.csv
  6. 종합분석_결과요약.csv

좌측 폴더 아이콘에서 다운로드 가능합니다!

다음 단계: 시각화 고도화, 특정 변수 집중 분석 등을 요청하세요! 💪
